In [1]:
function test_execute_query(mysqli $connection, string $query = "SELECT registerDate, username FROM nanohub.jos_users LIMIT 10;") 
{
    $result = $connection->query($query);
    while ($row = $result->fetch_row()) {
        print_r($row);
    }
}

In [2]:
class ConnectionConfig 
{
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

In [3]:
interface IMySqliConnection
{
    public function new_connection(ConnectionConfig $mysql_config);
}

class TunneledMySqliConnection implements IMySqliConnection
{
    public $ssh_config;
    public $local_port;
    
    public function __construct(
        ConnectionConfig $ssh_config, int $local_port = 3307
    )
    {
        $this->ssh_config = $ssh_config;
        $this->local_port = $local_port;
    }
    
    public function new_connection(ConnectionConfig $mysql_config)
    {    
        
        $ssh_connection_string = sprintf(
            "sshpass -p '%s' ssh -vfN -o 'StrictHostKeyChecking=no' -L %s:%s:%d %s@%s ", 
            $this->ssh_config->password, 
            $this->local_port,
            $mysql_config->hostname, 
            $mysql_config->port,
            $this->ssh_config->username,
            $this->ssh_config->hostname
        );
        
        shell_exec($ssh_connection_string);
        usleep(1000);
        
        mysqli_report(MYSQLI_REPORT_ERROR | MYSQLI_REPORT_STRICT);
        return mysqli_connect(
            $mysql_config->hostname, 
            $mysql_config->username,
            $mysql_config->password, 
            '',
            $this->local_port
        );
    }
}

class TunneledMySqliConnectionFactory
{
    static public function create_from_environment(int $local_port = 3307)
    {
        $ssh_config = new ConnectionConfig(
            getenv('tunnel_ssh_username'),
            getenv('tunnel_ssh_password'),
            getenv('tunnel_ssh_host'),
            getenv('tunnel_ssh_port')
        );
        
        $connection = new TunneledMySqliConnection($ssh_config, $local_port);
        
        $mysql_config = new ConnectionConfig(
            getenv('db_user'),
            getenv('db_password'),
            getenv('db_host'),
            getenv('tunnel_remote_bind_port')
        );
        
        return $connection->new_connection($mysql_config);
    }
    
}

In [6]:
$connection = TunneledMySqliConnectionFactory::create_from_environment(3318);

test_execute_query($connection);

Array
(
    [0] => 2014-11-13 21:09:09
    [1] => hubrepo
)



Array
(
    [0] => 
    [1] => admin
)



Array
(
    [0] => 2008-11-19 22:51:04
    [1] => support
)



Array
(
    [0] => 2008-11-18 17:29:56
    [1] => gridstat
)



Array
(
    [0] => 2008-11-11 19:17:04
    [1] => ncn
)



...

Array
(
    [0] => 2007-11-13 20:22:04
    [1] => repo
)



Array
(
    [0] => 2007-08-30 15:17:04
    [1] => violin
)



Array
(
    [0] => 2000-01-11 17:10:00
    [1] => mireles
)



Array
(
    [0] => 2000-07-05 07:07:09
    [1] => k_dadesh
)



null

In [6]:
$local_port = "3307";
// $tunnel = sprintf(
//     "nohup python -m sshtunnel -U saxenap -P %s -L :%s -R 127.0.0.1:3306 -p 22 db2.nanohub.org",
//     getenv('tunnel_ssh_password'),
//     $local_port
// );
$tunnel = sprintf(
    "sshpass -p '%s' ssh -vfN -o 'StrictHostKeyChecking=no' -L %s:%s:%d %s@%s ", 
    getenv('tunnel_ssh_password'), 
    $local_port,
    getenv('tunnel_remote_bind_address'), 
    getenv('tunnel_remote_bind_port'),
    getenv('tunnel_ssh_username'),
    getenv('tunnel_ssh_host')
);
$ssh_result = shell_exec($tunnel);
print_r($ssh_result);

// $connection = ssh2_connect(getenv('tunnel_ssh_host'), getenv('tunnel_ssh_port'));
// ssh2_auth_password($connection, getenv('tunnel_ssh_username'), getenv('tunnel_ssh_password'));
// ssh2_tunnel($connection, '127.0.0.1', 3306);



mysqli_report(MYSQLI_REPORT_ERROR | MYSQLI_REPORT_STRICT);
$db_connection = mysqli_connect(
    '127.0.0.1', 
    'saxenap_ro',
    getenv('db_password'), 
    'nanohub',
    $local_port
);

// $dsn = sprintf('mysql:host=%s;port=%s;dbname=nanohub', getenv('db_host'), $local_port);
// $db_connection = new PDO($dsn, 'saxenap_ro', getenv('db_password'));


$query = "SELECT registerDate, username FROM nanohub.jos_users LIMIT 10;";
$result = $db_connection->query($query);
while ($row = $result->fetch_row()) {
    print_r($row);
}

Array
(
    [0] => 2014-11-13 21:09:09
    [1] => hubrepo
)



Array
(
    [0] => 
    [1] => admin
)



Array
(
    [0] => 2008-11-19 22:51:04
    [1] => support
)



Array
(
    [0] => 2008-11-18 17:29:56
    [1] => gridstat
)



Array
(
    [0] => 2008-11-11 19:17:04
    [1] => ncn
)



Array
(
    [0] => 2014-06-26 19:38:57
    [1] => nanohub
)



Array
(
    [0] => 2007-11-13 20:22:04
    [1] => repo
)



Array
(
    [0] => 2007-08-30 15:17:04
    [1] => violin
)



Array
(
    [0] => 2000-01-11 17:10:00
    [1] => mireles
)



Array
(
    [0] => 2000-07-05 07:07:09
    [1] => k_dadesh
)



In [3]:
// $ssh['username'] = getenv('tunnel_ssh_username');
// $ssh['password'] = getenv('tunnel_ssh_password');
// $ssh['hostname'] = getenv('tunnel_ssh_host');
// $ssh['port'] = getenv('tunnel_ssh_port');

// $mysql['username'] = getenv('db_user');
// $mysql['password'] = getenv('db_password');
// $mysql['hostname'] = getenv('db_host');
// $mysql['port'] = getenv('tunnel_remote_bind_port');

In [4]:
$query = 'SELECT * FROM nanohub.jos_faq';

$ssh_query ='ssh -L 3307:' . getenv('tunnel_ssh_host').':' . getenv('tunnel_remote_bind_port').'; 
echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.getenv('db_user').' -h '.getenv('db_host').' --password='.getenv('db_password');

$response = ssh2_exec($connection, $ssh_query);
$error_response = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
print_r(stream_get_contents($error_response));

stream_set_blocking($response, true);
stream_set_blocking($error_response, true);

$results = stream_get_contents($response);
if (is_null($results ))
    echo "Empty response/result";

else 
    print_r($results);

ssh2_disconnect ( $connection );

PHP Warning:  ssh2_exec() expects parameter 1 to be resource, object given in /home/saxenap/.jupyter-php/pkgs/vendor/litipk/jupyter-php/src/Actions/ExecuteAction.php(115) : eval()'d code on line 4